In [13]:
import sqlite3
import requests
import datetime
import pandas as pd 

In [14]:
ACCESS_TOKEN = os.environ.get('ACCESS_TOKEN')

In [15]:
teams = {"Dodgers" : {"teamID": 1061, "venueID": 744},
         "Angels": {"teamID": 1062, "venueID" : 9763},
         "Giants" : {"teamID" : 197, "venueID" : 82}}

In [16]:
for team in teams:
    print(teams[team]["teamID"])

1061
1062
197


In [17]:
def get_ticket_info(teamID, venueID):
    url = f"https://api.stubhub.com/sellers/search/events/v3?venueId={venueID}&performerId={teamID}&parking=false&sort=eventDateLocal%20asc&rows=81"
    headers = { 
    "Authorization": "Bearer " + ACCESS_TOKEN,
    "Accept": "application/json"
    }
    r = requests.get(url, headers=headers)
    return r

def create_df(r):
    events = r.json()['events']
    prices = {event['id']:{
         "price": event['ticketInfo']['minPrice'],
         "time" : event['eventDateLocal'],
         "name" : event['name'],
         "extractTime":datetime.datetime.today(),
         "homeTeam" : event['performers'][0]['name'],
         "awayTeam" : event['performers'][1]['name'],
         "ticket_count": event["ticketInfo"]["totalTickets"]} for event in events}
    df = pd.DataFrame(prices).T.reset_index().rename({"index":"id"}, axis = 1)
    df["unique_name"] = df["name"] + ": " + df["time"].str[0:10]
    df['time'] = pd.to_datetime(df['time'].str[0:10])
    df['days_to_game'] = (df['time'] - df['extractTime'])
    df['days_to_game'] = df['days_to_game'].apply(lambda X: X.days)
    return df

In [11]:
for team in teams:
    teamID = teams[team]["teamID"]
    venueID = teams[team]["venueID"]
    r = get_ticket_info(teamID, venueID)
    df = create_df(r)
    with sqlite3.connect("../tickets.db") as conn: 
        df.to_sql("extraction", conn, if_exists = "append", index = False)

In [54]:
with sqlite3.connect("tickets.db") as conn: 
    cursor = conn.cursor()
    cursor.execute("SELECT sql FROM sqlite_master WHERE type='table';")

In [55]:
for result in cursor.fetchall():
    print(result[0])

In [49]:
with sqlite3.connect("../tickets.db") as conn: 
    cmd = "SELECT * FROM extraction"
    df2 = pd.read_sql_query(cmd, conn)

In [52]:
df2 = df2.drop(["time", "name", "homeTeam", "awayTeam", "unique_name"], axis = 1)

,id,price,extractTime,days_to_game,ticket_count
0,104722879,58.62,2021-06-08 17:51:48.284897,2,0
1,104725408,29.16,2021-06-08 17:51:48.284897,3,0
2,104725409,28.04,2021-06-08 17:51:48.284897,4,0
3,104724531,35.55,2021-06-08 17:51:48.284897,5,0
4,104725410,24.99,2021-06-08 17:51:48.284897,6,0
...,...,...,...,...,...
611,104738380,17.17,2021-06-10 12:10:34.477740,111,0
612,104736599,17.17,2021-06-10 12:10:34.477742,112,0
613,104736601,46.19,2021-06-10 12:10:34.477744,113,0
614,104738382,58.47,2021-06-10 12:10:34.477746,113,0


In [53]:
with sqlite3.connect("../tickets.db") as conn: 
    df2.to_sql("extraction", conn, if_exists = "replace", index = False)

In [11]:
df2.pivot_table(values = "price", columns = "days_to_game", index = "name")

days_to_game,-2,-1,0,1,2,3,4,5,6,7,...,107,108,109,110,111,112,113,114,115,116
name,,,,,,,,,,,,,,,,,,,,,
Arizona Diamondbacks at Los Angeles Dodgers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arizona Diamondbacks at San Francisco Giants,NaN,NaN,NaN,NaN,NaN,26.550,24.333333,23.97,27.984,22.870000,...,NaN,NaN,29.01,26.246667,21.0175,17.17,17.170000,NaN,NaN,NaN
Atlanta Braves at Los Angeles Dodgers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Atlanta Braves at San Francisco Giants,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Baltimore Orioles at Los Angeles Angels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Boston Red Sox at Los Angeles Angels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Chicago Cubs at Los Angeles Dodgers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Colorado Rockies at Los Angeles Angels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Colorado Rockies at Los Angeles Dodgers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Create Dates DF

In [42]:
r = get_ticket_info(teams["Dodgers"]["teamID"], teams["Dodgers"]["venueID"])
events = r.json()['events']
D = {event['id'] : {"time" : event['eventDateLocal'],
         "name" : event['name'],
         "homeTeam" : event['performers'][0]['name'],
         "awayTeam" : event['performers'][1]['name'],} for event in events}
games = pd.DataFrame(D).T.reset_index().rename({"index": "id"}, axis = 1)
games["unique_name"] = games["name"] + ": " + games["time"].str[0:10]
games['time'] = pd.to_datetime(games['time'].str[0:10])
games

,id,time,name,homeTeam,awayTeam,unique_name
0,104722879,2021-06-11,Texas Rangers at Los Angeles Dodgers,Los Angeles Dodgers,Texas Rangers,Texas Rangers at Los Angeles Dodgers: 2021-06-11
1,104725408,2021-06-12,Texas Rangers at Los Angeles Dodgers,Los Angeles Dodgers,Texas Rangers,Texas Rangers at Los Angeles Dodgers: 2021-06-12
2,104725409,2021-06-13,Texas Rangers at Los Angeles Dodgers,Los Angeles Dodgers,Texas Rangers,Texas Rangers at Los Angeles Dodgers: 2021-06-13
3,104724531,2021-06-14,Philadelphia Phillies at Los Angeles Dodgers,Los Angeles Dodgers,Philadelphia Phillies,Philadelphia Phillies at Los Angeles Dodgers: ...
4,104725410,2021-06-15,Philadelphia Phillies at Los Angeles Dodgers,Los Angeles Dodgers,Philadelphia Phillies,Philadelphia Phillies at Los Angeles Dodgers: ...
5,104725411,2021-06-16,Philadelphia Phillies at Los Angeles Dodgers,Los Angeles Dodgers,Philadelphia Phillies,Philadelphia Phillies at Los Angeles Dodgers: ...
6,104722883,2021-06-24,Chicago Cubs at Los Angeles Dodgers,Los Angeles Dodgers,Chicago Cubs,Chicago Cubs at Los Angeles Dodgers: 2021-06-24
7,104722884,2021-06-25,Chicago Cubs at Los Angeles Dodgers,Los Angeles Dodgers,Chicago Cubs,Chicago Cubs at Los Angeles Dodgers: 2021-06-25
8,104722885,2021-06-26,Chicago Cubs at Los Angeles Dodgers,Los Angeles Dodgers,Chicago Cubs,Chicago Cubs at Los Angeles Dodgers: 2021-06-26
9,104722886,2021-06-27,Chicago Cubs at Los Angeles Dodgers,Los Angeles Dodgers,Chicago Cubs,Chicago Cubs at Los Angeles Dodgers: 2021-06-27


In [48]:
with sqlite3.connect("../tickets.db") as conn: 
    games.to_sql("games", conn, if_exists = "replace", index = False)